In [1]:
# package 추가
import numpy as np
import pandas as pd

In [9]:
# 데이터 경로 설정
path_r = r'C:\Users\dlagh\Downloads\2020빅콘테스트 문제데이터(혁신아이디어분야)\02_카드매출데이터(신한카드)\CARD_SPENDING_RESIDENT\CARD_SPENDING_RESIDENT.txt'
path_f = r'C:\Users\dlagh\Downloads\2020빅콘테스트 문제데이터(혁신아이디어분야)\02_카드매출데이터(신한카드)\CARD_SPENDING_FOREIGNER\CARD_SPENDING_FOREIGNER.txt'

In [11]:
# 데이터 불러오기
data_r = pd.read_csv(path_r, encoding='utf-8', sep='\t')
data_f = pd.read_csv(path_f, encoding='unicode_escape', sep='\t')

In [15]:
data = data_r.append(data_f)
data

C:\Users\dlagh\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,AGE_CD,COUNTRY_NM,DONG_CD,GU_CD,MCT_CAT_CD,SEX_CD,STD_DD,USE_AMT,USE_CNT
0,25.0,NaN,517,110,10,F,20190201,595.0,11
1,50.0,NaN,517,110,10,F,20190201,243.0,5
2,55.0,NaN,517,110,10,F,20190201,27.0,5
3,20.0,NaN,517,110,10,M,20190201,243.0,11
4,25.0,NaN,517,110,10,M,20190201,433.0,11
...,...,...,...,...,...,...,...,...,...
238824,NaN,¿À½ºÆ®·¹ÀÏ¸®¾Æ,640,350,40,NaN,20200507,29.0,6
238825,NaN,¿À½ºÆ®·¹ÀÏ¸®¾Æ,612,350,40,NaN,20200512,15.0,3
238826,NaN,¿À½ºÆ®·¹ÀÏ¸®¾Æ,670,350,40,NaN,20200512,13.0,2
238827,NaN,¿À½ºÆ®·¹ÀÏ¸®¾Æ,640,350,71,NaN,20200516,39.0,1


In [17]:
# 카드 소비 카테고리
cate = data.MCT_CAT_CD.unique()
cate

array([10, 20, 21, 31, 32, 33, 34, 40, 52, 62, 70, 71, 80, 81, 92, 22, 30,
       35, 42, 44, 50, 43, 60], dtype=int64)

In [21]:
cate = [10, 20, 30, 32, 35, 52, 40, 42, 44, 80, 81]

In [25]:
# 데이터 롤링 & 비교일과의 업다운 라벨링
rolling_num = 1
compare_date = 7

# for c in cate:
data_c = data[data['MCT_CAT_CD'].isin(cate)]
data_g = data_c.groupby('STD_DD').sum()

# 2019 년
data_g_2019 = data_g[data_g.index<20200101]
dd = data_g_2019.rolling(rolling_num).sum()
d3_late = dd.USE_AMT[compare_date:].tolist()
additional = pd.DataFrame({'{}d_later'.format(rolling_num): d3_late})
dd.index = pd.to_datetime(dd.index, format="%Y%m%d")
dd['date'] = dd.index
dd = dd.reset_index(drop=True)
new = pd.concat([dd, additional], axis=1) 
final_data = new[['date','USE_AMT','{}d_later'.format(rolling_num)]].dropna()

final_data['ud'] = np.where(1.03*final_data['USE_AMT'] < final_data['{}d_later'.format(rolling_num)], 1, (np.where(final_data['USE_AMT'] > 1.03*final_data['{}d_later'.format(rolling_num)], 0, 10)))

#     final_data['ud'] = np.where((final_data['USE_AMT'] < final_data['3d_later']), 1, 0)
final_data.to_csv('code_{0}_2019_ud_rolling{1}.csv'.format("rf",rolling_num))

# 2020 년
data_g_2020 = data_g[data_g.index>=20200101]
dd = data_g_2020.rolling(rolling_num).sum()
d3_late = dd.USE_AMT[compare_date:].tolist()
additional = pd.DataFrame({'{}d_later'.format(rolling_num): d3_late})
dd.index = pd.to_datetime(dd.index, format="%Y%m%d")
dd['date'] = dd.index
dd = dd.reset_index(drop=True)
new = pd.concat([dd, additional], axis=1) 
final_data = new[['date','USE_AMT','{}d_later'.format(rolling_num)]].dropna()
final_data['ud'] = np.where(1.03*final_data['USE_AMT'] < final_data['{}d_later'.format(rolling_num)], 1, (np.where(final_data['USE_AMT'] > 1.03*final_data['{}d_later'.format(rolling_num)], 0, 10)))
final_data.to_csv('code_{0}_2020_ud_rolling{1}.csv'.format("rf", rolling_num))